In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import simpy
import random

In [2]:
class Customer:
    def __init__(self, id, arrival_time, is_vip=False):
        self.id = id
        self.arrival_time = arrival_time
        self.wait_time = 0
        self.service_time = 0
        self.is_vip = is_vip
    
    def __str__(self):
        vip_status = "VIP" if self.is_vip else "Regular"
        return f"Customer {self.id} ({vip_status}): Arrival={self.arrival_time:.2f}, Wait={self.wait_time:.2f}, Service={self.service_time:.2f}"

In [3]:
class BankSimulation:
    def __init__(self, num_tellers=3, mean_arrival_rate=0.1, mean_service_time=5, simulation_time=480, vip_probability=0.2, peak_hours=None, use_priority=True):
        self.env = simpy.Environment()
        self.num_tellers = num_tellers
        self.mean_arrival_rate = mean_arrival_rate  # customers per minute
        self.mean_service_time = mean_service_time  # minutes per customer
        self.simulation_time = simulation_time  # total simulation time in minutes
        self.vip_probability = vip_probability  # probability of a customer being VIP
        self.peak_hours = peak_hours or []  # list of (start_time, end_time, multiplier) tuples
        self.use_priority = use_priority  # whether to use priority queue
        
        # Create resources - use PriorityResource for priority queue
        if use_priority:
            self.tellers = simpy.PriorityResource(self.env, capacity=num_tellers)
        else:
            self.tellers = simpy.Resource(self.env, capacity=num_tellers)
        
        # Tracking
        self.customers = []
        self.waiting_times = []
        self.service_times = []
        self.vip_waiting_times = []
        self.regular_waiting_times = []
        self.teller_utilization = []
        self.utilization_times = []
        
    def customer_arrival(self):
        """Generate customer arrivals"""
        customer_id = 0
        while True:
            # Calculate current arrival rate based on peak hours
            current_rate = self.mean_arrival_rate
            current_time = self.env.now
            
            for start, end, multiplier in self.peak_hours:
                if start <= current_time < end:
                    current_rate *= multiplier
                    break
            
            # Generate next arrival
            interarrival_time = random.expovariate(current_rate)
            yield self.env.timeout(interarrival_time)
            
            # Create a new customer
            is_vip = random.random() < self.vip_probability
            customer = Customer(customer_id, self.env.now, is_vip)
            customer_id += 1
            
            self.customers.append(customer)
            self.env.process(self.serve_customer(customer))
    
    def serve_customer(self, customer):
        """Process for serving a customer"""
        arrival_time = self.env.now
        
        # Handle different resource types
        if self.use_priority:
            # For priority queue implementation
            priority = 0 if customer.is_vip else 1  # VIPs have higher priority (lower number)
            request = self.tellers.request(priority=priority)
        else:
            # For regular queue
            request = self.tellers.request()
        
        # Wait for a teller to become available
        yield request
        
        # Calculate waiting time
        customer.wait_time = self.env.now - arrival_time
        self.waiting_times.append(customer.wait_time)
        
        if customer.is_vip:
            self.vip_waiting_times.append(customer.wait_time)
        else:
            self.regular_waiting_times.append(customer.wait_time)
        
        # Generate service time
        customer.service_time = random.expovariate(1/self.mean_service_time)
        self.service_times.append(customer.service_time)
        
        # Serve the customer
        yield self.env.timeout(customer.service_time)
        
        # Release the teller
        self.tellers.release(request)
    
    def monitor_utilization(self):
        """Monitor teller utilization over time"""
        while True:
            self.utilization_times.append(self.env.now)
            self.teller_utilization.append(self.tellers.count / self.tellers.capacity)
            yield self.env.timeout(10)  # Check every 10 minutes
    
    def run(self):
        """Run the simulation"""
        # Start processes
        self.env.process(self.customer_arrival())
        self.env.process(self.monitor_utilization())
        
        # Run simulation
        self.env.run(until=self.simulation_time)
        
        return self.get_results()
    
    def get_results(self):
        """Return simulation results"""
        results = {
            'average_wait': np.mean(self.waiting_times) if self.waiting_times else 0,
            'max_wait': np.max(self.waiting_times) if self.waiting_times else 0,
            'average_service': np.mean(self.service_times) if self.service_times else 0,
            'customers_served': len(self.customers),
            'vip_average_wait': np.mean(self.vip_waiting_times) if self.vip_waiting_times else 0,
            'regular_average_wait': np.mean(self.regular_waiting_times) if self.regular_waiting_times else 0,
            'utilization_data': {
                'times': self.utilization_times,
                'values': self.teller_utilization
            }
        }
        return results

In [4]:
# 1. Change the number of tellers and analyze effect on waiting times
def task1_analyze_tellers():
    """Analyze the effect of different number of tellers on waiting times"""
    teller_counts = [1, 2, 3, 4, 5]
    results = []
    
    for num_tellers in teller_counts:
        sim = BankSimulation(num_tellers=num_tellers)
        result = sim.run()
        results.append({
            'num_tellers': num_tellers,
            'avg_wait': result['average_wait'],
            'max_wait': result['max_wait'],
            'customers_served': result['customers_served']
        })
    
    # Convert to DataFrame for analysis
    df = pd.DataFrame(results)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df['num_tellers'], df['avg_wait'], 'o-', label='Average Wait Time')
    plt.plot(df['num_tellers'], df['max_wait'], 's-', label='Maximum Wait Time')
    plt.xlabel('Number of Tellers')
    plt.ylabel('Wait Time (minutes)')
    plt.title('Effect of Number of Tellers on Customer Wait Times')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig('teller_analysis.png')
    plt.close()
    
    return df

In [5]:
# 2. Modify arrival rate to simulate peak hours
def task2_simulate_peak_hours():
    """Simulate peak hours with modified arrival rates"""
    # Define peak hours: (start_time, end_time, multiplier)
    peak_hours = [
        (60, 120, 2.5),   # Morning peak: 1-2 hours after opening, 2.5x normal rate
        (240, 300, 2.0)   # Lunch peak: 4-5 hours after opening, 2.0x normal rate
    ]
    
    # Run simulation with peak hours
    sim = BankSimulation(num_tellers=3, peak_hours=peak_hours)
    results = sim.run()
    
    # Generate time series of arrivals for visualization
    customer_arrivals = [c.arrival_time for c in sim.customers]
    
    # Plot histogram of arrivals
    plt.figure(figsize=(12, 6))
    plt.hist(customer_arrivals, bins=24, alpha=0.7)
    
    # Highlight peak hour periods
    for start, end, mult in peak_hours:
        plt.axvspan(start, end, alpha=0.3, color='red', label=f'{start}-{end} min: {mult}x rate')
    
    plt.xlabel('Simulation Time (minutes)')
    plt.ylabel('Number of Arrivals')
    plt.title('Customer Arrivals Throughout the Day with Peak Hours')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig('peak_hours_analysis.png')
    plt.close()
    
    return results

In [6]:
# 3. Visualize agent utilization over time
def task3_visualize_utilization():
    """Visualize teller utilization over time"""
    # Run simulation
    sim = BankSimulation(
        num_tellers=3,
        peak_hours=[
            (60, 120, 2.5),
            (240, 300, 2.0)
        ]
    )
    results = sim.run()
    
    # Get utilization data
    times = results['utilization_data']['times']
    util_values = results['utilization_data']['values']
    
    # Plot utilization
    plt.figure(figsize=(12, 6))
    plt.plot(times, util_values, '-o', markersize=4)
    plt.xlabel('Simulation Time (minutes)')
    plt.ylabel('Teller Utilization (fraction)')
    plt.title('Teller Utilization Over Time')
    plt.grid(True)
    
    # Highlight peak hours
    for start, end, mult in sim.peak_hours:
        plt.axvspan(start, end, alpha=0.2, color='red', label=f'Peak: {start}-{end} min')
    
    plt.ylim(0, 1.05)
    plt.legend()
    plt.tight_layout()
    plt.savefig('teller_utilization.png')
    plt.close()
    
    return times, util_values

In [7]:
# 4. Implement a priority queue for VIP customers
def task4_priority_queue():
    """Implement and analyze a priority queue for VIP customers"""
    # Compare regular queue vs priority queue
    results = []
    
    # Scenario 1: No priority queue (all customers equal)
    sim_regular = BankSimulation(num_tellers=3, vip_probability=0.2, use_priority=False)
    regular_results = sim_regular.run()
    
    # Scenario 2: With priority queue
    sim_priority = BankSimulation(num_tellers=3, vip_probability=0.2, use_priority=True)
    priority_results = sim_priority.run()
    
    # Combine results
    results = pd.DataFrame([
        {
            'scenario': 'Regular Queue',
            'vip_wait': regular_results['vip_average_wait'],
            'regular_wait': regular_results['regular_average_wait'],
            'overall_wait': regular_results['average_wait']
        },
        {
            'scenario': 'Priority Queue',
            'vip_wait': priority_results['vip_average_wait'],
            'regular_wait': priority_results['regular_average_wait'],
            'overall_wait': priority_results['average_wait']
        }
    ])
    
    # Plot comparison
    plt.figure(figsize=(10, 6))
    
    x = np.arange(2)
    width = 0.25
    
    plt.bar(x - width, [regular_results['vip_average_wait'], priority_results['vip_average_wait']], 
            width, label='VIP Customers', color='gold')
    plt.bar(x, [regular_results['regular_average_wait'], priority_results['regular_average_wait']], 
            width, label='Regular Customers', color='skyblue')
    plt.bar(x + width, [regular_results['average_wait'], priority_results['average_wait']], 
            width, label='Overall Average', color='lightgreen')
    
    plt.xlabel('Queue System')
    plt.ylabel('Average Wait Time (minutes)')
    plt.title('Comparison of Wait Times: Regular vs Priority Queue')
    plt.xticks(x, ['Regular Queue', 'Priority Queue'])
    plt.legend()
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig('priority_queue_analysis.png')
    plt.close()
    
    return results

In [8]:
# Main function to run all tasks
def main():
    print("Task 1: Analyzing the effect of different number of tellers")
    teller_analysis = task1_analyze_tellers()
    print(teller_analysis)
    print("\n" + "-"*50 + "\n")
    
    print("Task 2: Simulating peak hours")
    peak_results = task2_simulate_peak_hours()
    print(f"Peak hours results: {peak_results}")
    print("\n" + "-"*50 + "\n")
    
    print("Task 3: Visualizing teller utilization")
    task3_visualize_utilization()
    print("Utilization visualization saved")
    print("\n" + "-"*50 + "\n")
    
    print("Task 4: Implementing priority queue for VIP customers")
    priority_results = task4_priority_queue()
    print(priority_results)
    print("\n" + "-"*50 + "\n")

if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(42)
    random.seed(42)
    
    main()

Task 1: Analyzing the effect of different number of tellers
   num_tellers  avg_wait   max_wait  customers_served
0            1  5.426224  22.954870                43
1            2  0.158871   2.463975                42
2            3  0.000000   0.000000                44
3            4  0.000000   0.000000                56
4            5  0.000000   0.000000                48

--------------------------------------------------

Task 2: Simulating peak hours
Peak hours results: {'average_wait': np.float64(0.17361059625095765), 'max_wait': np.float64(3.8677786114424464), 'average_service': np.float64(6.779005162823207), 'customers_served': 54, 'vip_average_wait': np.float64(0.0), 'regular_average_wait': np.float64(0.19531192078232737), 'utilization_data': {'times': [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 4